# 1. Objet

Ce script retraite les données brutes enregistrées par le RawDataProcessor afin de :
- contrôler la conformité à certaines règles de gestion métier
- nettoyer les avoirs lorsque c'est possible
- agréger les informations à une maille commande
- calculer le canal majoritaire en poids
- effectuer d'autres aggrégations si nécessaire (ex : types de tarif...)

# 2. Imports et setup technique

In [1]:
from pathlib import Path
import sys
import datetime
import pandas as pd
from importlib import reload
project_root = str(Path(sys.path[0]).parents[0].absolute())
project_root
if project_root not in sys.path:
    sys.path.append(project_root)
import multiprocessing as mp
    
from scripts.utils import process_df  # traitement des avoirs
    
data_path = Path('..') / 'data'
persist_path = Path('..') / 'persist'
from IPython.display import display, HTML
display(HTML("<style>.container { width:90%; }</style>"))

from dask.distributed import LocalCluster, Client
import dask.dataframe as dd
client = Client(dashboard_address=':35000') # n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB'
client

Client Scheduler: tcp://127.0.0.1:41187 Dashboard: http://127.0.0.1:35000/status,Cluster Workers: 4 Cores: 8 Memory: 33.56 GB


In [2]:
orgacom_list = [
    '1ALO',
    '1BFC',
    '1CAP',
    '1CTR',
    '1EXP',
    '1LRO',
    '1LXF',
    '1NCH',
    '1OUE',
    '1PAC',
#     '1PLU', 
    '1PNO',
    '1PSU',
    '1RAA',
    '1SOU',
    '2BRE',
    '2CAE',
    '2CTR',
    '2EST',
    '2IDF',
#     '2IFC', Cash Européenne Food ?
    '2MPY',
    '2NOR',
    '2RAA',
    '2SES',
    '2SOU',
]

# Chargement des données brutes

In [3]:
%%time
with LocalCluster(
    n_workers=int(0.9 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    dashboard_address=':36000',
#     memory_limit='2GB',
#     ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client_:
    raw_data = dd.read_parquet(persist_path / 'raw_data.parquet')
    print(f'Il y a {len(raw_data)} lignes dans le dataset')

Il y a 229941217 lignes dans le dataset
CPU times: user 1.31 s, sys: 243 ms, total: 1.55 s
Wall time: 4.94 s


In [4]:
# timed : 6,45s

# %%time
# with LocalCluster(
#     n_workers=int(0.9 * mp.cpu_count()),
#     processes=False,
#     threads_per_worker=1,
#     dashboard_address=':36000',
# #     memory_limit='2GB',
# #     ip='tcp://localhost:9895',
# ) as cluster, Client(cluster) as client_:
#     raw_data = dd.read_parquet(persist_path / 'raw_data.parquet')
#     print(f'Il y a {len(raw_data)} lignes dans le dataset')

Il est nécessaire de recharger les catégories à la lecture du fichier parquet.
https://docs.dask.org/en/latest/dataframe-categoricals.html

In [5]:
for field in raw_data.dtypes.loc[lambda x: x == 'category'].index:
    raw_data[field] = raw_data[field].cat.set_categories(raw_data[field].head(1).cat.categories)

# Contrôle des données

On définit les fonctions qui vont permettre d'effectuer les contrôles.

### Contrôle des types de documents

In [6]:
valid_doctypes = ['ZC01', 'ZC02', 'ZC10'] # documents de type "commandes"
invalid_doctypes = ['ZR01', 'ZR02', 'ZA01', 'ZA02', 'ZA03', 'ZA04', 'ZC20']  # autres types de documents
doctypes = valid_doctypes + invalid_doctypes   

On s'assure qu'on n'a pas de type de document non prévu.

In [7]:
def ctrle_doctypes(data):
    if len(data.loc[~data.doctype.isin(doctypes)]) > 0:
        print(f'Unexpected doctypes have been encountered: '
              f'{data.loc[~data.doctype.isin(doctypes), "doctype"].value_counts().compute()}!')

In [8]:
%%time
with LocalCluster(
    n_workers=int(0.9 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    dashboard_address=':36000',
#     memory_limit='2GB',
#     ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client_:    
    ctrle_doctypes(raw_data)

Unexpected doctypes have been encountered: #      959834
ZB1       874
ZB5       435
Name: doctype, dtype: int64!
CPU times: user 16.8 s, sys: 1.83 s, total: 18.6 s
Wall time: 1min 56s


In [9]:
# timed to 8min 11s => mainly python code, should use processes.

# %%time
# with LocalCluster(
#     n_workers=int(0.9 * mp.cpu_count()),
#     processes=False,
#     threads_per_worker=1,
#     dashboard_address=':36000',
# #     memory_limit='2GB',
# #     ip='tcp://localhost:9895',
# ) as cluster, Client(cluster) as client_:    
#     ctrle_doctypes(raw_data)

### Contrôle des CA bruts non nuls alors que le poids est nul

D'un point de vue métier, il n'est pas possible qu'une ligne de **commande** (= valid_doctype) avec un poids nul ait un CA brut, sauf pour certains articles de service

Je veux : un rapport qui me sort, par succursale, les produits qui ont des lignes avec du revenue mais pas de poids, en comptant le nombre de lignes.

In [10]:
def ctrle_no_weight_revenue(data, order_doctypes=valid_doctypes):
    return(
        data.loc[
            data.doctype.isin(order_doctypes) &
            (data.weight == 0) &
            (data.brutrevenue != 0)
        ]
        .compute()
        .groupby(['orgacom', 'material'], observed=True)
        .size()
        .to_frame()
    )

In [11]:
%%time
with LocalCluster(
    n_workers=int(0.9 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    dashboard_address=':36000',
#     memory_limit='2GB',
#     ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client_:
    result = ctrle_no_weight_revenue(raw_data)

CPU times: user 9.46 s, sys: 1.13 s, total: 10.6 s
Wall time: 1min 2s


In [12]:
# timed : 4min 14s

# %%time
# with LocalCluster(
#     n_workers=int(0.9 * mp.cpu_count()),
#     processes=False,
#     threads_per_worker=1,
#     dashboard_address=':36000',
# #     memory_limit='2GB',
# #     ip='tcp://localhost:9895',
# ) as cluster, Client(cluster) as client_:
#     result = ctrle_no_weight_revenue(raw_data)

In [13]:
with pd.option_context('display.max_columns', None):
    display(result.unstack('orgacom', fill_value=0).sort_index(axis=1).style.bar(align='mid', axis=None))

L'essentiel des articles concernés sont des articles de service (forfaits livraison). Il faudrait regarder ce qui s'est passé sur le 156486, il remonte régulièrement côté ES.

### Contrôle des origines de commande

In [14]:
# def ctrle_origin(data, valid_origins=valid_origins):
#     report = data.loc[
#         ~data.origin.isin(valid_origins) &
#         (data.brutrevenue != 0) &
#         data.doctype.isin(valid_doctypes)
#     ].value_counts('origin').rename('counts').to_frame()
#     report['percentage'] = report['counts'] / sum(
#         (data.brutrevenue != 0) &
#         data.doctype.isin(valid_doctypes)
#     )
#     return(report.loc[report['counts'] > 0])

In [15]:
valid_origins = ['TV', 'VR', 'WEB', 'EDI']

def ctrle_origin(data, valid_origins=valid_origins):
    sizes = (
        data.loc[
            (data.brutrevenue != 0) &
            data.doctype.isin(valid_doctypes)
        ].groupby(['orgacom'], observed=True).size()
        .compute()
    ).rename('baseline')
    
    invalid_origins = (
        data.loc[
            (data.brutrevenue != 0) &
            data.doctype.isin(valid_doctypes) &
            ~data.origin.isin(valid_origins)
        ].groupby(['orgacom', 'origin'], observed=True).size()
        .compute()    
    ).rename('invalid').to_frame()
    invalid_origins = invalid_origins.reset_index().merge(sizes.reset_index(), on='orgacom', how='left')
    invalid_origins['percentage'] = invalid_origins['invalid'] / invalid_origins['baseline']
    return(invalid_origins)

In [16]:
%%time
with LocalCluster(
    n_workers=int(0.9 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    dashboard_address=':36000',
#     memory_limit='2GB',
#     ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client:
    display(ctrle_origin(raw_data).sort_values(['orgacom', 'origin']).style.format({'percentage': lambda x: f'{x:.2%}'}).bar(subset=['percentage'], align='mid', ))

,orgacom,origin,invalid,baseline,percentage
9,1ALO,#,21339,14019888,0.15%
42,1ALO,TELE,5,14019888,0.00%
32,1ALO,DFUE,3,14019888,0.00%
13,1BFC,#,4151,7625418,0.05%
0,1CAP,#,2320,301484,0.77%
12,1CAP,SCHR,98,301484,0.03%
18,1CTR,#,5598,7428760,0.08%
35,1CTR,DFUE,39,7428760,0.00%
27,1EXP,#,124,52145,0.24%
23,1LRO,#,4549,7062511,0.06%


CPU times: user 15.6 s, sys: 1.81 s, total: 17.4 s
Wall time: 2min 8s


In [17]:
# timed: 8min 59s...

# %%time
# with LocalCluster(
#     n_workers=int(0.9 * mp.cpu_count()),
#     processes=False,
#     threads_per_worker=1,
#     dashboard_address=':36000',
# #     memory_limit='2GB',
# #     ip='tcp://localhost:9895',
# ) as cluster, Client(cluster) as client:
#     display(ctrle_origin(raw_data).sort_values(['orgacom', 'origin']).style.format({'percentage': lambda x: f'{x:.2%}'}).bar(subset=['percentage'], align='mid', ))

### Retraitement des avoirs

In [ ]:
from scripts.utils import process_df

In [28]:
process_df?

In [37]:
%%time

with LocalCluster(n_workers=int(0.9 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    dashboard_address=':36000',
) as cluster, Client(cluster) as client:
    temp_ddf = raw_data.map_partitions(
        lambda x: process_df(
            x.reset_index(),
            orders_doctypes=['ZC10'],
            avoirs_doctypes=['ZA01', 'ZA02'],
        ).set_index('date').drop('_duplicated'),
        meta=raw_data,
    )

CPU times: user 250 ms, sys: 341 ms, total: 591 ms
Wall time: 2.26 s


In [39]:
temp_ddf.get_partition(50).compute()

KeyError: "['_duplicated'] not found in axis"

In [ ]:
def process_credit(
    data,
    orders_doctypes=['ZC10'],
    credit_doctypes=['ZA01', 'ZA02'],
    indicators=['margin', 'brutrevenue', 'weight'],
#     grouper_fields=['orgacom', 'date', 'client', 'material'],
):
    grouper_fields = ['date', 'orgacom', 'client', 'material']
    indicator_fields = ['weight', 'brutrevenue', 'margin']

#     credits = data.loc[data.doctype.isin(credit_doctypes)].compute()
#     credits = credits.groupby(grouper_fields, observed=True)[indicator_fields].sum()
    credits = 'skipped'
    
    orders = data.loc[data.doctype.isin(orders_doctypes)]
    orders = orders.groupby(grouper_fields, observed=True).size()
    
    return(credits, orders)

In [ ]:
raw_data.get_partition(400).compute()

In [ ]:
(client.cluster.scheduler_info)

In [ ]:
toto, tata = process_credit(raw_data.get_partition(400))
display(toto)
display(tata.loc[lambda x: x>1].compute())

In [ ]:
def credit_processing(data):
    len_before = len(data.loc[data.doctype.isin(['ZA01', 'ZA02'])])
    brutrev_before = data.loc[data.doctype.isin(['ZA01', 'ZA02']), 'brutrevenue'].sum()
    print('Avant nettoyage')
    print(f"Nombre d'avoirs ZA01 et ZA02 : {len_before}")
    print(f"CA représenté par ces avoirs : {brutrev_before: .2f} €")
    temp = process_df(data, ).drop('_duplicated', axis=1)
    len_after = len(temp.loc[temp.doctype.isin(['ZA01', 'ZA02'])])
    brutrev_after = temp.loc[temp.doctype.isin(['ZA01', 'ZA02']), 'brutrevenue'].sum()    
    print('Après nettoyage')
    print(f"Nombre d'avoirs ZA01 et ZA02 : {len_after}")
    print(f"CA représenté par ces avoirs : {brutrev_after: .2f} €")  
    return(temp)

In [ ]:
def process_credit(
    group,
    orders_doctypes=['ZC10'],
    credit_doctypes=['ZA01', 'ZA02'],
    indicators=['margin', 'brutrevenue', 'weight'],
#     grouper_fields=['orgacom', 'date', 'client', 'material'],
):
    if (
        (sum(group.doctype.isin(orders_doctypes)) == 1) &
        (group.loc[group.doctype.isin(orders_doctypes + credit_doctypes), ['brutrevenue', 'weight']].sum() >= 0).all()
    ):
        return(pd.DataFrame([[0., 1., 2.]] * len(group), columns=indicators, index=group.index))
    else:
        return(group[indicators])

In [ ]:
test_df = pd.read_pickle(persist_path / 'rawbyoc' / 'data_1SOU.pkl')

In [ ]:
test_df = test_df.loc[(test_df.date >= '2019-01-01') & (test_df.date < '2019-02-28')]
test_df

In [ ]:
%%time
zboubi = test_df.groupby(['date', 'client', 'material']).apply(process_credit)

In [ ]:
zboubi.loc[(zboubi.margin == 0.) & (zboubi.brutrevenue == 1.)]

In [ ]:
test_df.loc[7402238:7402245]

### Application du nettoyage, des filtres, et aggrégation

On va boucler sur chacun des fichier, procéder au nettoyage des accords, appliquer les filtres, puis calculer l'aggrégation. A la fin, on concatène les dataframes aggrégés dans un unique dataframe qu'on persistera.

In [ ]:
pd.read_pickle(persist_path / 'rawbyoc' / 'data_1SOU.pkl')

In [ ]:
4192726/ 19083945

In [ ]:
len(df)

In [ ]:
sum(df.index.duplicated())

In [ ]:
df = data
grouper_fields = ['orgacom', 'date', 'client', 'material']
indicators = ['margin', 'brutrevenue', 'weight']
orders_doctypes=['ZC10']
avoirs_doctypes=['ZA01', 'ZA02']


mask_ZC = df.doctype.isin(['ZC10'])
mask_ZA = df.doctype.isin(['ZA01', 'ZA02'])
raw_avoirs = df.loc[mask_ZA, grouper_fields + indicators]
avoirs = raw_avoirs.groupby(grouper_fields, observed=True).sum()
mask_dup_ZC = (df.loc[mask_ZC]
             .duplicated(grouper_fields, keep=False)
             .rename('_duplicated'))
df = df.merge(
    mask_dup_ZC,
    how='left',
    left_index=True,
    right_index=True)
df['_duplicated'] = df['_duplicated'].fillna(False)
print('ici')
to_update = (
    df.loc[~df._duplicated & mask_ZC, grouper_fields + indicators]
    .merge(avoirs,
           how='inner',
           left_on=grouper_fields,
           right_index=True,
           validate='1:1')
)
for indicator in indicators:
    to_update[indicator] = (to_update[indicator + '_x'] +
                            to_update[indicator + '_y'])
to_update = to_update.loc[(to_update.weight >= 0) &
                          (to_update.brutrevenue >= 0)]
to_update.drop(columns=[indicator + '_x' for indicator in indicators] +
                       [indicator + '_y' for indicator in indicators],
               inplace=True)
mask_to_del = (
    df.set_index(grouper_fields)
      .index.isin(to_update.set_index(grouper_fields).index)
)
df = df.loc[~mask_to_del | ~df.doctype.isin(avoirs_doctypes)]

In [ ]:
aggfuncs = {
    'margin': 'sum', 
    'brutrevenue': 'sum',
    'weight': 'sum',
    'origin': 'size',
}

order_groupers = ['orgacom', 'date', 'client', 'origin']  # pour les commandes, on considèrera ces clefs de regroupement
order_df_list = []  # aggrégé par date, client, canal, orgacom 
pricetype_groupers = ['orgacom', 'date', 'client', 'origin', 'pricetype_init', 'pricetype_applied']
pricetype_df_list = []


for orgacom in orgacom_list:
    print('----------------------------------------------------------------')
    print(f'{datetime.datetime.now()} - Début du traitement pour {orgacom}')
    print('----------------------------------------------------------------')
    print(f'{datetime.datetime.now()} - Lecture du fichier')
    data = pd.read_pickle(persist_path / 'rawbyoc' / f'data_{orgacom}.pkl')
    print(f'{datetime.datetime.now()} - Traitement des avoirs')
    data = credit_processing(data)
    print(f'{datetime.datetime.now()} - Application des filtres: CA > 0, origine de commande ok, poids > 0')
    data = data.loc[
        data.origin.isin(valid_origins) & 
        (data.brutrevenue > 0) &
        (data.weight > 0)
    ]
    print(f"{datetime.datetime.now()} - Calcul de l'aggrégation 'orders'")
    order_df = data.groupby(order_groupers, observed=True).agg(aggfuncs).rename({'origin': 'linecount'}, axis=1)
    order_df_list.append(order_df)
#     print(f"{datetime.datetime.now()} - Calcul de l'aggrégation 'pricetype'")
#     pricetype_df = data.groupby(pricetype_groupers, observed=True).agg(aggfuncs).rename({'origin': 'linecount'}, axis=1)
#     pricetype_df_list.append(pricetype_df)

In [ ]:
pd.concat(order_df_list, axis=0).to_pickle(persist_path / 'orders_all_SV.pkl')

# Calcul du canal majoritaire en poids

In [ ]:
raw_orders = pd.read_pickle(persist_path / 'orders_all_SV.pkl')

In [ ]:
print(f'Nb orders in initial dataset: {len(raw_orders)}')
target_len = len(raw_orders.reset_index().loc[:, ['orgacom', 'client', 'date']].drop_duplicates())
print(f'Target order count in order dataset: {target_len}')

In [ ]:
%%time
main_origin = (
    raw_orders['weight']
    .unstack('origin', fill_value=0)
    .idxmax(axis=1)
    .rename('main_origin')
)

In [ ]:
orders_with_main_origin = (
raw_orders
    .unstack('origin', fill_value=0)
    .rename_axis(('indicator', 'origin'), axis=1)
    .groupby('indicator', axis=1).sum()
).join(main_origin)

On vérifie que le calcul du canal majoritaire n'a pas modifié les indicateurs au total :

In [ ]:
check = orders_with_main_origin.iloc[:, :-1].join(raw_orders.groupby(['orgacom', 'date', 'client'], observed=True).sum(), rsuffix='_r')
for indicator in ['margin', 'brutrevenue', 'weight', 'linecount']:
    check[indicator + '_delta'] = (check[indicator] - check[indicator + '_r']).abs()
    assert(sum(check[indicator + '_delta']) == 0)
del(check)

In [ ]:
orders_with_main_origin.to_pickle(persist_path / 'orders_all_SV_with_main_origin.pkl')